# COLLECTING IMDBIDS AND TITLES
---

Nhiệm vụ phần này: từ danh sách các đường dẫn phim của trang web `Just Watch`, thực hiện cào thông tin **imdbId** và **title** trong đường dẫn nếu không có id và lưu lại vào file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [1]:
import time
import os
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re

## 2. CÀO THÔNG TIN IMDBID TRONG TRANG `JUST WATCH`

In [2]:
links = []

with open('../Data/links.txt', 'r') as f:
    links = [line.strip() for line in f]

Dùng selenium thoi vì request k được

In [3]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "start": 0,
        "end": 628
    },

    "Thi": {
        "part": 2,
        "start": 629,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "start": 1259,
        "end": 1887
    } 
}

In [4]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
start = info["start"]
end = info["end"]

In [5]:
def collect_imdbID_title(links, start, end):
    
    option = webdriver.ChromeOptions()

    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.managed_default_content_settings.plugins": 2,
        "profile.managed_default_content_settings.media": 2
    }
    option.add_experimental_option("prefs", prefs)
    # option.page_load_strategy = 'eager'
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    imdbIds_titles = []

    for url in links[start:end + 1]:
        print(f"[LOG]: parse imdbid of url {url}")
        driver.get(url)
        time.sleep(3)
        page_source = driver.page_source

        imdb_id_matches = re.findall(r'"imdbId":"(tt\d+)"', page_source)
        if imdb_id_matches:
            imdb_id = imdb_id_matches[-1]
            imdbIds_titles.append(imdb_id)
            print("IMDB ID:", imdb_id)
        else:
            # print("Không tìm thấy IMDb ID.")
            match = re.search(r'class="release-year".*?\((\d{4})\)', page_source)
            release_year = match.group(1)
            movie_title_meta = driver.find_element(By.XPATH, "//meta[@property='og:title']")
            if movie_title_meta:
                movie_title = movie_title_meta.get_attribute("content").split(" streaming")[0]
                movie_title = f"{movie_title} __ {release_year}"
            else:
                movie_title = "Null"
            imdbIds_titles.append(movie_title)
            print(movie_title)

    driver.quit()
    return imdbIds_titles

In [ ]:
# khuyến nghị mạng yếu: chia phần mình thành các phần nhỏ hơn rồi collect, rùi thêm vào file nhé
imdbIds_titles = collect_imdbID_title(links, start=1801, end=1887)
imdbIds_titles

## 3. LƯU THÔNG TIN VỀ ID VÀ TITLE VÀO FILE TXT

In [17]:
file_name = f"../Data/ids_titles_part{part}.csv"

with open(file_name, 'a') as f:
    for item in imdbIds_titles:
        f.write(item + "\n")